In [1]:
# Load modules and set-up test problem
import sys
# sys.path.append("..")

import numpy as np
import matplotlib.pyplot as plt

import cuqi
from cuqi.testproblem import Deconvolution1D
from cuqi.distribution import Gaussian, Gamma, JointDistribution
from cuqi.implicitprior import RegularizedGaussian, RegularizedGMRF, RegularizedUniform
from cuqi.sampler import RegularizedLinearRTO, Gibbs, Conjugate

# Set seed
np.random.seed(42)

num_samples = 100
num_burnin = 10

print("Pre-implemented constraints: ", RegularizedGaussian.constraint_options())
print("Pre-implemented regularization: ", RegularizedGaussian.regularization_options())

Pre-implemented constraints:  ['nonnegativity', 'box']
Pre-implemented regularization:  ['l1', 'TV']


# Overview of various models and features for Regularized Gaussians within CUQIpy

# Nonnegativity constraints with Gibbs

In [ ]:
n = 128
A, y_data, info = Deconvolution1D(dim=n, phantom='square').get_components()

l = Gamma(1, 1e-4)
d = Gamma(1, 1e-4)
x = RegularizedGMRF(mean = np.zeros(n), prec = lambda d : d, constraint="nonnegativity")
y = Gaussian(A@x, prec = lambda l : l)

# Combine into a joint distribution
joint = JointDistribution(l, d, x, y)
posterior = joint(y=y_data)

sampling_strategy = {
    'x': RegularizedLinearRTO,
    'l': Conjugate,
    'd': Conjugate
}


sampler = Gibbs(posterior, sampling_strategy)
samples = sampler.sample(num_samples, num_burnin)

samples['l'].plot_trace()
samples['d'].plot_trace()
plt.figure()
samples['x'].plot_ci()
plt.plot(info.exactSolution)

plt.figure()
samples['x'].plot_ci_width()

# L1 regularization

In [ ]:
n = 128
A, y_data, info = Deconvolution1D(dim=n, phantom='square').get_components()

x = RegularizedUniform(regularization="l1", strength = 10, geometry=A.domain_geometry)
y = Gaussian(A@x, prec = 1000)

# Combine into a joint distribution
joint = JointDistribution(x, y)
posterior = joint(y=y_data)

sampler = RegularizedLinearRTO(posterior,
                                maxit=400,      
                                stepsize=100) 
samples = sampler.sample(num_samples, num_burnin)


plt.figure()
samples.plot_ci()
plt.plot(info.exactSolution)

plt.figure()
samples.plot_ci_width()

In [ ]:
n = 128
A, y_data, info = Deconvolution1D(dim=n, phantom='square').get_components()

x = RegularizedGMRF(mean = np.zeros(n), prec = 10, regularization="l1", strength = 10, geometry=A.domain_geometry)
y = Gaussian(A@x, prec = 1000)

# Combine into a joint distribution
joint = JointDistribution(x, y)
posterior = joint(y=y_data)

sampler = RegularizedLinearRTO(posterior,
                                maxit=100,      
                                stepsize=10,
                                adaptive=True) 
samples = sampler.sample(num_samples, num_burnin)


plt.figure()
samples.plot_ci()
plt.plot(info.exactSolution)

plt.figure()
samples.plot_ci_width()

# L1 regularization with Gibbs

In [ ]:
# Wrapper to tune the parameters
class _RegularizedLinearRTO(RegularizedLinearRTO):
    def __init__(self, target):
        super().__init__(target, x0=None, maxit=100, stepsize = 10.0, adaptive = True)


n = 128
A, y_data, info = Deconvolution1D(dim=n, phantom='square').get_components()


l = Gamma(1, 1e-4)
d = Gamma(1, 1e0) # Very strong hyper prior to stabilize the algorithm
x = RegularizedUniform(regularization = "l1", strength = lambda d : np.sqrt(d), geometry = A.domain_geometry)
y = Gaussian(A@x, prec = lambda l : l)

# Combine into a joint distribution
joint = JointDistribution(l, d, x, y)
posterior = joint(y=y_data)

sampling_strategy = {
    'l': Conjugate,
    'd': Conjugate,
    'x': _RegularizedLinearRTO
}


sampler = Gibbs(posterior, sampling_strategy)
samples = sampler.sample(100, 50)

samples['l'].plot_trace()
samples['d'].plot_trace()
plt.figure()
samples['x'].plot_ci()
plt.plot(info.exactSolution)

plt.figure()
samples['x'].plot_ci_width()

In [ ]:
# Wrapper to tune the parameters
class _RegularizedLinearRTO(RegularizedLinearRTO):
    def __init__(self, target):
        super().__init__(target, x0=None, maxit=200, stepsize = 0.01)


n = 128
A, y_data, info = Deconvolution1D(dim=n, phantom='square').get_components()


l = Gamma(1, 1e-4)
d = Gamma(1, 1e0)
x = RegularizedGMRF(mean = np.zeros(n), prec = lambda d : 0.1*d,
                    regularization = "l1", strength = lambda d : np.sqrt(d),
                    geometry = A.domain_geometry)
y = Gaussian(A@x, prec = lambda l : l)

# Combine into a joint distribution
joint = JointDistribution(l, d, x, y)
posterior = joint(y=y_data)

sampling_strategy = {
    'x': _RegularizedLinearRTO,
    'd': Conjugate,
    'l': Conjugate
}


sampler = Gibbs(posterior, sampling_strategy)
samples = sampler.sample(100, 10)

samples['l'].plot_trace()
samples['d'].plot_trace()
plt.figure()
samples['x'].plot_ci()
plt.plot(info.exactSolution)

plt.figure()
samples['x'].plot_ci_width()

# Pre-implemented TV

In [2]:
n = 128
A, y_data, info = Deconvolution1D(dim=n, phantom='square').get_components()

x = RegularizedUniform(regularization="tv", strength = 10, geometry=A.domain_geometry)
y = Gaussian(A@x, prec = 1000)

# Combine into a joint distribution
joint = JointDistribution(x, y)
posterior = joint(y=y_data)

sampler = RegularizedLinearRTO(posterior,
                                maxit=100,      
                                stepsize=10,
                                adaptive=True) 
samples = sampler.sample(num_samples, num_burnin)


plt.figure()
samples.plot_ci()
plt.plot(info.exactSolution)

plt.figure()
samples.plot_ci_width()

c:\Users\jmev\Documents\GitHub\CUQIpy\cuqi\distribution\_gaussian.py:673: RuntimeWarning: divide by zero encountered in log
  logdet = np.sum(-np.log(precision))


Sample 12 / 110

KeyboardInterrupt: 

In [ ]:
# Wrapper to tune the parameters
class _RegularizedLinearRTO(RegularizedLinearRTO):
    def __init__(self, target):
        super().__init__(target, x0=None, maxit=100, stepsize = 10.0, adaptive = True)


n = 128
A, y_data, info = Deconvolution1D(dim=n, phantom='square').get_components()


l = Gamma(1, 1e-4)
d = Gamma(1, 1e0) # Very strong hyper prior to stabilize the algorithm
x = RegularizedUniform(regularization = "TV", strength = lambda d : np.sqrt(d), geometry = A.domain_geometry)
y = Gaussian(A@x, prec = lambda l : l)

# Combine into a joint distribution
joint = JointDistribution(l, d, x, y)
posterior = joint(y=y_data)

sampling_strategy = {
    'l': Conjugate,
    'd': Conjugate,
    'x': _RegularizedLinearRTO
}


sampler = Gibbs(posterior, sampling_strategy)
samples = sampler.sample(100, 50)

samples['l'].plot_trace()
samples['d'].plot_trace()
plt.figure()
samples['x'].plot_ci()
plt.plot(info.exactSolution)

plt.figure()
samples['x'].plot_ci_width()

# Custom TV

In [ ]:
from cuqi.operator import FirstOrderFiniteDifference
fd = np.array(FirstOrderFiniteDifference(n-1, bc_type='zero').get_matrix().todense()).T

from cuqi.solver import ProximalL1, ProjectNonnegative

x  = RegularizedGaussian(0.5*np.ones(n), 1000, proximal = [(lambda x, scale: ProximalL1(x, 3*scale), fd), (lambda x, scale : ProjectNonnegative(x), np.eye(n))])
y  = Gaussian(A@x, 0.001)

geom = x.geometry

joint = JointDistribution(x, y)
posterior = joint(y=y_data)

sampler = RegularizedLinearRTO(posterior,
                                maxit=100,      
                                stepsize=1) 
samples = sampler.sample(100)

plt.figure()
samples.plot_median() # Median is a more representative point-estimate for this distribution
samples.plot_ci()
plt.plot(info.exactSolution)
plt.legend(["median", "CI", "mean", "truth"])
plt.show()

plt.figure()
samples.plot_ci_width()
plt.show()

plt.figure()
plt.plot(samples.samples[:,10])
plt.plot(samples.samples[:,25])
plt.plot(samples.samples[:,40])
plt.show()

In [ ]:
from cuqi.operator import FirstOrderFiniteDifference
fd = np.array(FirstOrderFiniteDifference(n-1, bc_type='zero').get_matrix().todense()).T

from cuqi.solver import ProximalL1, ProjectNonnegative

x  = RegularizedUniform(proximal = [(lambda x, scale: ProximalL1(x, 3*scale), fd), (lambda x, scale : ProjectNonnegative(x), np.eye(n))],
                        geometry = A.domain_geometry)
y  = Gaussian(A@x, 0.001)

geom = x.geometry

joint = JointDistribution(x, y)
posterior = joint(y=y_data)

sampler = RegularizedLinearRTO(posterior,
                                maxit=100,      
                                stepsize=10) 
samples = sampler.sample(100)

plt.figure()
samples.plot_median() # Median is a more representative point-estimate for this distribution
samples.plot_ci()
plt.plot(info.exactSolution)
plt.legend(["median", "CI", "mean", "truth"])
plt.show()

plt.figure()
samples.plot_ci_width()
plt.show()

plt.figure()
plt.plot(samples.samples[:,10])
plt.plot(samples.samples[:,25])
plt.plot(samples.samples[:,40])
plt.show()